# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Linear Reg Project').master('local[4]').getOrCreate()

22/06/14 10:44:18 WARN Utils: Your hostname, mehrdad-Standard resolves to a loopback address: 127.0.1.1; using 128.179.176.230 instead (on interface wlp4s0)
22/06/14 10:44:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/14 10:44:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/06/14 10:44:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/06/14 10:44:20 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/06/14 10:44:20 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/06/14 10:44:20 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/06/

In [3]:
data = spark.read.csv('../data/cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
# age, tonnage, passengers, length, cabins, passenger_densitiy are all numeric attributes
# we need to convert cruise_line to one_hot
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [6]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

### Convert Cruise_line to Categorical

In [7]:
# how many unique cruise lines do we have?
data.select('Cruise_line').distinct().count()

20

In [8]:
# so we will encode the strings to allow for Linear Regression

In [9]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_indices')
fit_indexer = indexer.fit(data)

In [10]:
data_2 =fit_indexer.transform(data)

### Create Data Frame with Feature Vector and Label

In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
inputCols = ['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'Cruise_line_indices']
outputCol = 'features'
assembler = VectorAssembler(inputCols=inputCols, outputCol=outputCol)

In [13]:
assembler.transform(data_2)

DataFrame[Ship_name: string, Cruise_line: string, Age: int, Tonnage: double, passengers: double, length: double, cabins: double, passenger_density: double, crew: double, Cruise_line_indices: double, features: vector]

In [14]:
data_3 = assembler.transform(data_2)

In [15]:
final_data = data_3.select('features', 'crew')

In [16]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



### Create Train / Test Sets

In [17]:
train, test = final_data.randomSplit([0.7, 0.3])

### Train Linear Regression Model

In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
lr = LinearRegression(featuresCol='features', labelCol='crew')

In [20]:
lr_model = lr.fit(train)

22/06/14 10:44:33 WARN Instrumentation: [8f308a12] regParam is zero, which might cause numerical instability and overfitting.


In [21]:
predictions = lr_model.evaluate(test)

In [22]:
predictions.r2

0.9417162422305447

In [23]:
predictions.rootMeanSquaredError

0.8152596474227782

In [24]:
predictions.meanAbsoluteError

0.6459857613668659